# Draft

For this third draft, I will:

-Ultimately create a single loop for which lists are added for each variable 

-In that loop, Search "info" for either Multi-Family or Acres first and "pass" for that listing  

-For the third div w/ beds/baths/sqft/garage, update the way you grab each list item

-Once that loop is created and final table is assembled, look into doing same for other URLs

In [2]:
import requests
from bs4 import BeautifulSoup # web scraper
import re # regex which will be used for the third div
import pandas as pd # for dataframe assembly

# This is copied from Albon Notes

# Create a variable with the url
url = 'https://www.coldwellbankerhomes.com/ma/brookline/'

# Use requests to get the contents
r = requests.get(url)

# Get the text of the contents
html_content = r.text

# Convert the html content into a beautiful soup object
cb = BeautifulSoup(html_content, 'lxml') # cb = Coldwell Banker

Prelim exploration can go here (i.e. how many listings, div discussion etc)

In [3]:
propinfo = cb.select(".prop-info")

In [4]:
print(propinfo[0].text.strip())

689 Washington 
#1 
Brookline, MA 02445
$749,000


Condo
Active
MLS # 72561426
Updated 2 hours ago

3Beds
1Full Bath
1Partial Bath
2Total Baths
1,097Sq. Ft.

Listed by Jill Streck (617) 510-0771




HideContactShareMap


In [5]:
addprice = cb.select(".address-price-etc") # addresses and prices
descrip = cb.select(".description-summary") # just going to get the property type from this one
bbsg = cb.select(".highlights") # beds, baths, sq ft, garage

In [6]:
for i in range (7,10):
    print(i)
    print(addprice[i].text.strip()) # first div
    print()
    print(descrip[i].text.strip()) # second div
    print()
    print(bbsg[i].text.strip()) # third div
    print() 

7
135 Crafts Rd 
Brookline, MA 02467
$3,750,000

Single Family
Active
MLS # 72564092
Updated 9 hours ago

5Beds
3Full Baths
2Partial Baths
5Total Baths
4,175Sq. Ft.
2Car Garage

8
133 Columbia St 
Brookline, MA 02446
$2,600,000

Multi-Family
Active
MLS # 72564224
Updated 9 hours ago

2Units
8Total Beds
5Total Baths
0.15Acres
2Car Garage

9
133 Columbia St 
#2 
Brookline, MA 02446
$1,499,000

Duplex
Active
MLS # 72564223
Updated 9 hours ago

5Beds
2Full Baths
1Partial Bath
3Total Baths
2,312Sq. Ft.
1Car Garage



Basically, as you can see above, the 9th (listed here as 8; Python starts from 0) listing on CB's page is a Multi-Family listing that includes "Units" and "Acres" instead of the standard "Square Feet." I was tempted to include these but some of these Multi-Family listings don't even list beds. Thus, we will *exclude* these listings. 

In [7]:
# First div classes we need for address and price
staddress = cb.select(".street-address")
prices_ = cb.select(".price")
# Second div classes
proptype = cb.select(".description-summary li")
# Third div
bbsg = cb.select(".highlights") # beds, baths, sq ft, garage

# will be putting this in the next block for looped urls

We wanna do this all in one for loop so it will be easier to automate for the other pages of listings. 

In [36]:
beds = []
tbaths = []
types = []
addresses = []
prices = []
sqft = []
garages = []
unit = "Unit"
index = 0
for i in range(24):
    info3_ = bbsg[i].text # getting info for third div
    info3 = info3_.replace(',','') # deleting the ',' in sqft value
    if unit in info3: # ignoring listings that are units
        pass
    else:
        # first div
        addresses.append(staddress[i].text.strip())
        prices.append(prices_[i].text.strip()) # Prices done
        # second div
        types.append(proptype[4*i].text.strip()) # Property types done
        # third div
        bed = re.findall(r"(\d+)Bed", info3)
        beds = beds + bed # Beds done
        tbath = re.findall(r"(\d+)Total Bath", info3)
        tbaths = tbaths + tbath # Total baths done
        nft = re.findall(r"(\d+)Sq", info3)
        sqft = sqft + nft # Sqft done
        ngarages = re.findall(r"(\d+)Car Garage", info3)
        if ngarages:
            pass
        else:
            ngarages = ['0'] # some listings don't have garages
        garages = garages + ngarages # Garages done

In [34]:
test = pd.DataFrame({'Addresses': addresses, 
                     'Price': prices, 
                     'Property Type': types, 
                     'Beds': beds, 
                     'Baths': tbaths, 
                     'Square Feet': sqft, 
                     'Garages': garages})

In [35]:
test

,Addresses,Price,Property Type,Beds,Baths,Square Feet,Garages
0,689 Washington,"$749,000",Condo,3,2,1097,0
1,90 Lawton St,"$1,580,000",Condo,3,3,1936,1
2,255 Walnut St,"$1,299,000",Condo,4,2,2533,1
3,359 Tappan St,"$999,000",Condo,3,2,1638,0
4,20 Chapel,"$879,000",Condo,2,2,945,1
5,14 Still St,"$1,049,000",Townhouse,1,3,1790,0
6,439 Chestnut Hill Ave,"$5,490,000",Single Family,8,7,7500,3
7,135 Crafts Rd,"$3,750,000",Single Family,5,5,4175,2
8,133 Columbia St,"$1,499,000",Duplex,5,3,2312,1
9,133 Columbia St,"$1,299,000",Condo,3,2,1524,1


The moment of truth: dare we try to get this for other pages?

Algorithm for URLs page 2 through 6:

In [79]:
test2 = pd.DataFrame

In [100]:
for j in range (2,7):
    print(j)

2
3
4
5
6


In [104]:
for j in range(2,6):
    num = 24*j
    url = 'https://www.coldwellbankerhomes.com/ma/brookline/?sortId=2&offset=' + str(num)
    print(url)
    # Setting up
    r = requests.get(url)
    html_content = r.text
    cb = BeautifulSoup(html_content, 'lxml') 
    
    # Getting HTML
    # First div classes we need for address and price
    staddress = cb.select(".street-address")
    prices_ = cb.select(".price")
    # Second div classes
    proptype = cb.select(".description-summary li")
    # Third div classes
    bbsg = cb.select(".highlights") # beds, baths, sq ft, garage
    
    # Performing the loop
    beds = []
    tbaths = []
    types = []
    addresses = []
    prices = []
    sqft = []
    garages = []
    unit = "Unit"
    for i in range(24):
        info3_ = bbsg[i].text # getting info for third div
        info3 = info3_.replace(',','') # deleting the ',' in sqft value
        if unit in info3: # ignoring listings that are units
            pass
        else:
            # first div
            addresses.append(staddress[i].text.strip())
            prices.append(prices_[i].text.strip()) # Prices done
            # second div
            types.append(proptype[4*i].text.strip()) # Property types done
            # third div
            bed = re.findall(r"(\d+)Bed", info3)
            beds = beds + bed # Beds done
            tbath = re.findall(r"(\d+)Total Bath", info3)
            tbaths = tbaths + tbath # Total baths done
            nft = re.findall(r"(\d+)Sq", info3)
            sqft = sqft + nft # Sqft done
            ngarages = re.findall(r"(\d+)Car Garage", info3)
            if ngarages:
                pass
            else:
                ngarages = ['0'] # some listings don't have garages
            garages = garages + ngarages # Garages done
    df = pd.DataFrame({'Addresses': addresses, 
                     'Price': prices, 
                     'Property Type': types, 
                     'Beds': beds, 
                     'Baths': tbaths, 
                     'Square Feet': sqft, 
                     'Garages': garages})
    test = test.append(df)

https://www.coldwellbankerhomes.com/ma/brookline/?sortId=2&offset=48


IndexError: list index out of range

In [96]:
beds

['4',
 '3',
 '3',
 '6',
 '2',
 '5',
 '4',
 '2',
 '3',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '4',
 '2',
 '1',
 '1',
 '5',
 '3',
 '6',
 '6']

In [97]:
df

,Addresses,Price,Property Type,Beds,Baths,Square Feet,Garages
0,110 Arlington Rd,"$1,599,000",Single Family,4,3,2280,2
1,28 Alton Pl,"$1,250,000",Condo,3,3,1554,1
2,166 Walnut St,"$629,000",Condo,3,1,1134,1
3,197 Cabot,"$8,500,000",Single Family,6,8,6720,3
4,640 Hammond St,"$1,449,000",Condo,2,3,2272,2
5,348 Buckminster Rd,"$4,200,000",Single Family,5,5,4917,2
6,5 Grassmere Rd,"$1,399,000",Single Family,4,3,2290,2
7,630 Hammond St,"$1,399,000",Condo,2,2,1515,2
8,29 Green St,"$1,225,000",Condo,3,2,1393,1
9,33 Pond Ave,"$850,000",Condo,2,2,1005,2


In [103]:
test

,Addresses,Price,Property Type,Beds,Baths,Square Feet,Garages
0,689 Washington,"$749,000",Condo,3,2,1097,0
1,90 Lawton St,"$1,580,000",Condo,3,3,1936,1
2,255 Walnut St,"$1,299,000",Condo,4,2,2533,1
3,359 Tappan St,"$999,000",Condo,3,2,1638,0
4,20 Chapel,"$879,000",Condo,2,2,945,1
5,14 Still St,"$1,049,000",Townhouse,1,3,1790,0
6,439 Chestnut Hill Ave,"$5,490,000",Single Family,8,7,7500,3
7,135 Crafts Rd,"$3,750,000",Single Family,5,5,4175,2
8,133 Columbia St,"$1,499,000",Duplex,5,3,2312,1
9,133 Columbia St,"$1,299,000",Condo,3,2,1524,1


In [ ]:
 df = df.append({'Addresses': addresses[i], 
                     'Price': prices[i], 
                     'Property Type': types[i], 
                     'Beds': beds[i], 
                     'Baths': tbaths[i], 
                     'Square Feet': sqft[i], 
                     'Garages': garages[i]}, ignore_index=True)

In [81]:
test2

pandas.core.frame.DataFrame

In [78]:
test

,Addresses,Price,Property Type,Beds,Baths,Square Feet,Garages
0,689 Washington,"$749,000",Condo,3,2,1097,0
1,90 Lawton St,"$1,580,000",Condo,3,3,1936,1
2,255 Walnut St,"$1,299,000",Condo,4,2,2533,1
3,359 Tappan St,"$999,000",Condo,3,2,1638,0
4,20 Chapel,"$879,000",Condo,2,2,945,1
5,14 Still St,"$1,049,000",Townhouse,1,3,1790,0
6,439 Chestnut Hill Ave,"$5,490,000",Single Family,8,7,7500,3
7,135 Crafts Rd,"$3,750,000",Single Family,5,5,4175,2
8,133 Columbia St,"$1,499,000",Duplex,5,3,2312,1
9,133 Columbia St,"$1,299,000",Condo,3,2,1524,1


Got first and second pages. Problem we're having with third page is that property type is no long every 4 list elements of its div because some properties don't have an "updated x days ago" blurb.